#Usando GAN para gerar novos digitos

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [2]:
os.environ["KERAS_BACKEND"] = "tensorflow"

np.random.seed(42)
random_dim = 100

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5  
X_train = X_train.reshape(60000, 784)

In [4]:
otimizador = Adam(lr=0.0002, beta_1=0.5)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [5]:
gerador = Sequential()
gerador.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
gerador.add(LeakyReLU(0.2))

gerador.add(Dense(512))
gerador.add(LeakyReLU(0.2))

gerador.add(Dense(1024))
gerador.add(LeakyReLU(0.2))

gerador.add(Dense(784, activation='tanh'))
gerador.compile(loss='binary_crossentropy', optimizer=otimizador)

In [6]:
discriminador = Sequential()
discriminador.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminador.add(LeakyReLU(0.2))
discriminador.add(Dropout(0.3))

discriminador.add(Dense(512))
discriminador.add(LeakyReLU(0.2))
discriminador.add(Dropout(0.3))

discriminador.add(Dense(256))
discriminador.add(LeakyReLU(0.2))
discriminador.add(Dropout(0.3))

discriminador.add(Dense(1, activation='sigmoid'))
discriminador.compile(loss='binary_crossentropy', optimizer=otimizador)

In [7]:
def gerar_gan(discriminador, randomDim, gerador, otimizador):
  discriminador.trainable = False
  gan_input = Input(shape=(random_dim,))
  x = gerador(gan_input)

  gan_output = discriminador(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=otimizador)

  return gan

In [8]:
def salvar_digito_gerado(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    imagem_gerada = gerador.predict(noise)
    imagem_gerada = imagem_gerada.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(imagem_gerada.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(imagem_gerada[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_digito_gerado_epoca_%d.png' % epoch)

In [ ]:
epochs = 400
batchSize = 128

gLosses = []
dLosses = []

batch_count = X_train.shape[0] / batchSize

generator = gerador
gan = gerar_gan(discriminador, random_dim, gerador, otimizador)

batchCount = X_train.shape[0] / batchSize
print('Épocas:', epochs)
print('Batch size:', batchSize)
print('Batches por época:', batchCount)

for e in range(1, epochs+1):
    print('-'*15, 'Época %d' % e, '-'*15)

    noise = np.random.normal(0, 1, size=[batchSize, random_dim])
    imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

    imagemGerada = gerador.predict(noise)
    X = np.concatenate([imageBatch, imagemGerada])

    yDis = np.zeros(2*batchSize)
    yDis[:batchSize] = 0.9

    discriminador.trainable = True
    dloss = discriminador.train_on_batch(X, yDis)

    yGen = np.ones(batchSize)
    discriminador.trainable = False
    gloss = gan.train_on_batch(noise, yGen)

    gLosses.append(gloss)
    dLosses.append(dloss)

    salvar_digito_gerado(e)

Épocas: 400
Batch size: 128
Batches por época: 468.75
--------------- Época 1 ---------------
4/4 [==============================] - 0s 15ms/step
--------------- Época 2 ---------------
4/4 [==============================] - 0s 13ms/step
--------------- Época 3 ---------------
4/4 [==============================] - 0s 16ms/step
--------------- Época 4 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 5 ---------------
4/4 [==============================] - 0s 6ms/step
--------------- Época 6 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 7 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 8 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 9 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 10 ---------------
4/4 [==============================] - 0s 8ms/step
--------------- Época 11 -------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


--------------- Época 22 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 23 ---------------
4/4 [==============================] - 0s 6ms/step
--------------- Época 24 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 25 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 26 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 27 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 28 ---------------
4/4 [==============================] - 0s 8ms/step
--------------- Época 29 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 30 ---------------
4/4 [==============================] - 0s 7ms/step
--------------- Época 31 ---------------
4/4 [==============================] - 0s 6ms/step
--------------- Época 32 ---------------
4/4 [==============================] - 

In [1]:
gerador.save('GAN_gerador_MNIST.h5')
discriminador.save('GAN_discriminador_MNIST.h5')

NameError: ignored